In [1]:
from itertools import chain

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from tqdm import tqdm


import torch
import spacy
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
# initialize hyper hyper parameters
torch.manual_seed(420)
np.random.seed(420)

# Load data

In [3]:
fallacies_df = pd.read_csv('../data/fallacies.csv', index_col=0)
approved_df = pd.read_csv('../data/approved.csv', index_col=0)

In [4]:
df = pd.concat([
    fallacies_df,
    approved_df[approved_df.n_supporters >= 5].drop("n_supporters", axis=1)
])
df.fallacy_reason = df.fallacy_reason.fillna('')
df = df[~df.premise_content.isna()]

In [5]:
vc = df.fallacy_type.value_counts()

In [6]:
df = df[df.fallacy_type.isin(vc.head(10).index)]

# Text preprocessing

In [7]:
nlp = spacy.load('en_core_web_sm')

In [8]:
def preprocess_sentence(sent):
    sent = sent.lower()
    sent = nlp(sent)
    words = map(lambda x: x.text, sent)
    return list(words)

In [9]:
df['premise_content_preprocessed'] = df.premise_content.apply(preprocess_sentence)

In [10]:
df = df.sample(frac=1, random_state=420).reset_index(drop=True) # shuffle the data

In [11]:
train_df, test_df = train_test_split(df, test_size=0.1, random_state=420)

In [12]:
word_vocab = {"<oov>", "<pad>"}
word_vocab = word_vocab.union(
    set(chain.from_iterable(map(lambda x: x[1]["premise_content_preprocessed"], train_df.iterrows())))
)
word_to_ix = {word: i for i, word in enumerate(word_vocab)}

In [13]:
fallacy_vocab = sorted(list(set(df.fallacy_type.unique())))
fallacy_to_ix = {word: i for i, word in enumerate(fallacy_vocab)}

# Neural Net

In [14]:
class Net(nn.Module):
    def __init__(self, word_vocab_size, word_embedding_dim, fallacy_vocab_size, max_sent_size):
        super(Net, self).__init__()
        
        self.word_embeddings = nn.Embedding(
            word_vocab_size, word_embedding_dim
        )  # random init

        hidden = 100
        self.fc1 = nn.Linear(word_embedding_dim, hidden)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden * max_sent_size, fallacy_vocab_size)
        
    def forward(self, word_inputs):
        word_embeds = self.word_embeddings(word_inputs)
        h1 = self.fc1(word_embeds)
        a1 = self.relu(h1)
        h2 = self.fc2(a1.view(a1.shape[0], -1))
        return h2

In [15]:
MAX_SENT_SIZE = 100

In [16]:
def pad_and_convert_to_ints(data):
    X = np.full((len(data), MAX_SENT_SIZE), word_to_ix["<pad>"])

    for i, (_, x) in enumerate(data.iterrows()):
        X[i, :len(x["premise_content_preprocessed"])] = [
            (word_to_ix[word] if word in word_to_ix else word_to_ix['<oov>'])
            for word in x["premise_content_preprocessed"]
        ]

    return X

In [17]:
trainX = pad_and_convert_to_ints(train_df)
testX = pad_and_convert_to_ints(test_df)

In [18]:
trainY = train_df.fallacy_type.apply(lambda x: fallacy_to_ix[x]).values
testY = test_df.fallacy_type.apply(lambda x: fallacy_to_ix[x]).values

In [19]:
net = Net(
    len(word_vocab),
    300,
    len(fallacy_vocab),
    MAX_SENT_SIZE,
)
opt = optim.Adam(net.parameters(), lr=0.001, betas=(0.9, 0.999))
criterion = nn.CrossEntropyLoss()

In [20]:
def train_epoch(X, Y, model, opt, criterion, batch_size=50):
    model.train()
    losses = []
    for beg_i in range(0, X.shape[0], batch_size):
        x_batch = X[beg_i : beg_i + batch_size, :]
        y_batch = Y[beg_i : beg_i + batch_size]
        x_batch = torch.tensor(x_batch)
        y_batch = torch.tensor(y_batch)

        opt.zero_grad()

        y_pred = model(x_batch)

        loss = criterion(y_pred, y_batch)

        loss.backward()
        
        opt.step()

        losses.append(loss.data.numpy())

    return [sum(losses) / float(len(losses))]

In [21]:
e_losses = []
num_epochs = 50
for e in tqdm(range(num_epochs)):
    e_losses += train_epoch(trainX, trainY, net, opt, criterion, batch_size=100)

100%|██████████| 50/50 [00:57<00:00,  1.09s/it]


In [22]:
e_losses

[1.648647062918719,
 1.1579338803010828,
 0.8639095863875221,
 0.6519683020956376,
 0.49754232957082634,
 0.3916252329945564,
 0.3144337480120799,
 0.26024389179313884,
 0.22472450838369482,
 0.2021918112740797,
 0.1836471551043146,
 0.16652311954428167,
 0.15389170563396284,
 0.15304387185503454,
 0.1436967558282263,
 0.14165554579128237,
 0.13380510666791132,
 0.13742583508000655,
 0.12878754932214231,
 0.12868005935760105,
 0.12467504183159155,
 0.12622744973529787,
 0.11861445009708405,
 0.1246930636903819,
 0.11598835775957388,
 0.12288909931393231,
 0.11224004746798207,
 0.1150429095415508,
 0.10807052999734879,
 0.11504645579878021,
 0.10653888543739039,
 0.11149197751108338,
 0.10753859864438281,
 0.11131234160240959,
 0.10827652616974186,
 0.11072050692404017,
 0.10511275315109421,
 0.10995837542064049,
 0.10358512478277963,
 0.10620599875555319,
 0.10325937071705565,
 0.1080795708386337,
 0.09915090396123774,
 0.10312960121561499,
 0.0985911726294195,
 0.10133734082474428,
 0

# Metrics (macro f1 score)

In [23]:
with torch.no_grad():
    net.eval()
    x = torch.tensor(testX)
    y_pred = net(x)

In [24]:
score = f1_score(testY, y_pred.numpy().argmax(axis=1), average='macro')

/usr/local/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [25]:
score

0.19663854875283446

In [26]:
for i in range(10):
    print("Guessing " + fallacy_vocab[i] + "\t for all:", end='\t')
    print(f1_score(testY, np.array( [i]*testY.shape[0]), average='macro'))

Guessing Appeal To Authority	 for all:	0.004347826086956523
Guessing Appeal To Belief	 for all:	0.0064516129032258064
Guessing Begging The Question	 for all:	0.005405405405405406
Guessing Fallacy Of False Cause	 for all:	0.00851063829787234
Guessing Fallacy Of Red Herring	 for all:	0.005405405405405406
Guessing Irrelevant Conclusion	 for all:	0.02178217821782178
Guessing None	 for all:	0.07959866220735787
Guessing Poisoning The Well	 for all:	0.0011049723756906078
Guessing Prejudicial Language	 for all:	0.003278688524590164
Guessing Wrong Direction	 for all:	0.0074866310160427805


In [27]:
print("Random Guessing:", end='\t')
print(f1_score(testY, np.random.randint(0,10,testY.shape[0]), average='macro'))

Random Guessing:	0.05702413272128415
